In [ ]:
#! /usr/bin/env python

import requests, csv, time
MAX_RETRIES = 20

# In : Gidsvoornederland.nl URL to a public library infopage : e.g. https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=199891
# Out: URL to Wayback Machine (The Internet Archive - TIA) snapshot: e.g. https://web.archive.org/web/20181111190849/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=199891 
# Works for html pages in  Gidsvoornederland.nl
# This script is certainly not 100% solid & error proof, you might need to tweak it here and there to make it more robust and deal with all sorts of possible errors and time-outs. But hey, it does a mostly decent job!

def read_local_csv(filename):
    # Input=local filepath/filename of csv -- Output=List
    list = []
    response = open(filename, 'r', encoding="utf8")
    sheet = csv.reader(response, delimiter=';')
    for row in sheet:
        if not str(row).startswith('#', 2):  # skip rows in cvs that start with '#'
            list.append(row)
    return list

base_url = 'http://web.archive.org'

urllist = []
urllist=read_local_csv("Input-Gids_TeArchiverenURLs_09102019_werkbestand.txt") # file with 1 to-be-archived Leesplein-URL per line

with open("Output-Gids_TeArchiverenURLs_09102019_werkbestand.txt", "w") as outputfile: # open new versin of outputfile, clear the old version

    errorfile = open('Output-Gids_TeArchiverenURLs_09102019_werkbestand.txt', 'a') # deze file bevat alle LP-urls die in de 1e run niet gearchiveerd konden worden

    for url in urllist:
        time.sleep(5)  # pause for 5 sec, not to overrequest the server. Choose longer times for archiving pdf and doc files, as the archiving software needs more time to read these docs than standard html pages
        outputfile = open('Output-Gids_TeArchiverenURLs_09102019_werkbestand.txt', 'a') # append to existing outputfile
        linenumber = urllist.index(url)+1
        #print(str(url[0]))

        #https://stackoverflow.com/questions/33895739/python-requests-cant-load-any-url-remote-end-closed-connection-without-respo
        url_wbm = base_url + '/save/' + str(url[0])
        session = requests.Session()
        adapter = requests.adapters.HTTPAdapter(max_retries=MAX_RETRIES)
        session.mount('https://', adapter)
        session.mount('http://', adapter)

        r = session.get(url_wbm)  #This line actually saves to URL to the Waybackmachine. All the following lines of code are for administration/display
        print(str(url[0])+ " -- " + str(r.headers))

# r.headers fout die kan voorkomen: {'Server': 'nginx/1.13.11', ...., 'X-Archive-Wayback-Runtime-Error': 'WaybackException: Unexpected Error', 'Set-Cookie': 'JSESSIONID=9EAAD47DBDCB85D0514293D472995036; Path=/; HttpOnly', 'X-App-Server': 'wwwb-app53', 'X-ts': '----'}

# Als deze fout voorkomt en de LvdL-URL daardoor niet gearchiveerd wordt: Schrijf niet gearchiveerde LvdL-url weg naar apart bestand - dat kunnen we dan later als invoer voor een correctierun gebruiken

        if 'X-Archive-Wayback-Runtime-Error' in r.headers:
            errorfile.write(str(url[0]) + "\n")
        else: #r.headers bevat geen Error
            #archive_url ="" #only relevant for pdf and doc
            #archive_url= r.headers['X-Cache-Key']
            # X-Cache-Key for PDF and DOC: httpweb.archive.org/web/20180608110403/https://www.leesplein.nl/assets/juryrapporten/2015-schaduw.pdfNL
            # X-Cache-Key for HTML: httpweb.archive.org/save/https://www.leesplein.nl/JB_plein.php?hm=3&sm=1&leefcat=12+&id=6726NL

            #if archive_url.startswith('httpweb.') and archive_url.endswith('NL'):
                #archive_url = "http://"+archive_url[4:-2]
                # for PDF and DOC : http://web.archive.org/web/20180608110403/https://www.leesplein.nl/assets/juryrapporten/2015-schaduw.pdf
                # for HTML: http://web.archive.org/save/https://www.leesplein.nl/JB_plein.php?hm=3&sm=1&leefcat=12+&id=6726
            #else: print('Unexpected value of X-Cache-Key in r.headers')

            if r.status_code == requests.codes.ok:
                contenttype = r.headers['Content-Type']
                if "text/html" in contenttype: #html page -- X-Cache-Key is not relevant
                    print(str(linenumber) + "^^^^" + str(url[0])+ "^^^^" + base_url + r.headers['Content-Location']) # the "^^^^" is used a separator between the two URLs. The more traditional separators such as ";" or "," are not reliable enough, as the can occur in URLs.
                    outputfile.write(str(linenumber) + "^^^^" + str(url[0])+ "^^^^" + base_url + r.headers['Content-Location'] + "\n")
                #elif contenttype == "application/pdf": #pdf file
                    #print(str(linenumber) + "^^^^" + str(url[0]) + "^^^^" + archive_url)
                    #outputfile.write(str(linenumber) + "^^^^" + str(url[0]) + "^^^^" + archive_url + "\n")
                #elif contenttype == "application/vnd.openxmlformats-officedocument.wordprocessingml.document": #Word doc(x)
                    #print(str(linenumber) + "^^^^" + str(url[0]) + "^^^^" + archive_url)
                    #outputfile.write(str(linenumber) + "^^^^" + str(url[0]) + "^^^^" + archive_url + "\n")
                else:
                    print("Unknown Content-Type: " + str(contenttype))
            else:
                print('Error in response: ' + str(r.status_code))
        outputfile.close()
        #errorfile.close()

https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5296448 -- {'Server': 'nginx/1.15.8', 'Date': 'Wed, 09 Oct 2019 10:39:34 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Location': '/web/20191009103933/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5296448', 'Set-Cookie': 'JSESSIONID=572112AE0987E8BAC611260176092D54; Path=/; HttpOnly', 'X-Archive-Orig-Vary': 'Accept-Encoding', 'X-Archive-Guessed-Charset': 'UTF-8', 'X-Archive-Orig-Pragma': 'no-cache', 'X-Archive-Orig-Server': 'Apache', 'X-Archive-Orig-Connection': 'close', 'X-Archive-Orig-Content-Type': 'text/html; charset=UTF-8', 'X-Archive-Orig-Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'X-Archive-Orig-Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'X-Archive-Orig-Date': 'Wed,

https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5296471 -- {'Server': 'nginx/1.15.8', 'Date': 'Wed, 09 Oct 2019 10:40:09 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Location': '/web/20191009104008/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5296471', 'Set-Cookie': 'JSESSIONID=B16CC16C0DC9673FF0DC969AF045E175; Path=/; HttpOnly', 'X-Archive-Orig-Vary': 'Accept-Encoding', 'X-Archive-Guessed-Charset': 'UTF-8', 'X-Archive-Orig-Pragma': 'no-cache', 'X-Archive-Orig-Server': 'Apache', 'X-Archive-Orig-Connection': 'close', 'X-Archive-Orig-Content-Type': 'text/html; charset=UTF-8', 'X-Archive-Orig-Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'X-Archive-Orig-Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'X-Archive-Orig-Date': 'Wed,

https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5296452 -- {'Server': 'nginx/1.15.8', 'Date': 'Wed, 09 Oct 2019 10:40:40 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Location': '/web/20191009104039/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5296452', 'Set-Cookie': 'JSESSIONID=8993E0CCE40446C1F79B996528C05CBB; Path=/; HttpOnly', 'X-Archive-Orig-Vary': 'Accept-Encoding', 'X-Archive-Guessed-Charset': 'UTF-8', 'X-Archive-Orig-Pragma': 'no-cache', 'X-Archive-Orig-Server': 'Apache', 'X-Archive-Orig-Connection': 'close', 'X-Archive-Orig-Content-Type': 'text/html; charset=UTF-8', 'X-Archive-Orig-Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'X-Archive-Orig-Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'X-Archive-Orig-Date': 'Wed,

https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5296541 -- {'Server': 'nginx/1.15.8', 'Date': 'Wed, 09 Oct 2019 10:41:18 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Location': '/web/20191009104116/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5296541', 'Set-Cookie': 'JSESSIONID=90B83DF9737D95229140EB9114CB0F21; Path=/; HttpOnly', 'X-Archive-Orig-Vary': 'Accept-Encoding', 'X-Archive-Guessed-Charset': 'UTF-8', 'X-Archive-Orig-Pragma': 'no-cache', 'X-Archive-Orig-Server': 'Apache', 'X-Archive-Orig-Connection': 'close', 'X-Archive-Orig-Content-Type': 'text/html; charset=UTF-8', 'X-Archive-Orig-Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'X-Archive-Orig-Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'X-Archive-Orig-Date': 'Wed,

https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5296473 -- {'Server': 'nginx/1.15.8', 'Date': 'Wed, 09 Oct 2019 10:41:49 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Location': '/web/20191009104148/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5296473', 'Set-Cookie': 'JSESSIONID=CECBEB9966B65080CD04C8B4442B5A95; Path=/; HttpOnly', 'X-Archive-Orig-Vary': 'Accept-Encoding', 'X-Archive-Guessed-Charset': 'UTF-8', 'X-Archive-Orig-Pragma': 'no-cache', 'X-Archive-Orig-Server': 'Apache', 'X-Archive-Orig-Connection': 'close', 'X-Archive-Orig-Content-Type': 'text/html; charset=UTF-8', 'X-Archive-Orig-Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'X-Archive-Orig-Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'X-Archive-Orig-Date': 'Wed,

...

https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5369438 -- {'Server': 'nginx/1.15.8', 'Date': 'Wed, 09 Oct 2019 11:30:51 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Location': '/web/20191009113050/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5369438', 'Set-Cookie': 'JSESSIONID=5CD713E823F2B6BB24FC6147C26A67D7; Path=/; HttpOnly', 'X-Archive-Orig-Vary': 'Accept-Encoding', 'X-Archive-Guessed-Charset': 'UTF-8', 'X-Archive-Orig-Pragma': 'no-cache', 'X-Archive-Orig-Server': 'Apache', 'X-Archive-Orig-Connection': 'close', 'X-Archive-Orig-Content-Type': 'text/html; charset=UTF-8', 'X-Archive-Orig-Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'X-Archive-Orig-Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'X-Archive-Orig-Date': 'Wed,

https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=1933350 -- {'Server': 'nginx/1.15.8', 'Date': 'Wed, 09 Oct 2019 11:31:26 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Location': '/web/20191009113125/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=1933350', 'Set-Cookie': 'JSESSIONID=A92EA4806C6D075972D8F41C909E6555; Path=/; HttpOnly', 'X-Archive-Orig-Vary': 'Accept-Encoding', 'X-Archive-Guessed-Charset': 'UTF-8', 'X-Archive-Orig-Pragma': 'no-cache', 'X-Archive-Orig-Server': 'Apache', 'X-Archive-Orig-Connection': 'close', 'X-Archive-Orig-Content-Type': 'text/html; charset=UTF-8', 'X-Archive-Orig-Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'X-Archive-Orig-Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'X-Archive-Orig-Date': 'Wed,

https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5317208 -- {'Server': 'nginx/1.15.8', 'Date': 'Wed, 09 Oct 2019 11:31:58 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Location': '/web/20191009113157/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5317208', 'Set-Cookie': 'JSESSIONID=195DCD92C5C05BD5A81864C731C183FC; Path=/; HttpOnly', 'X-Archive-Orig-Vary': 'Accept-Encoding', 'X-Archive-Guessed-Charset': 'UTF-8', 'X-Archive-Orig-Pragma': 'no-cache', 'X-Archive-Orig-Server': 'Apache', 'X-Archive-Orig-Connection': 'close', 'X-Archive-Orig-Content-Type': 'text/html; charset=UTF-8', 'X-Archive-Orig-Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'X-Archive-Orig-Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'X-Archive-Orig-Date': 'Wed,

https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5310959 -- {'Server': 'nginx/1.15.8', 'Date': 'Wed, 09 Oct 2019 11:32:31 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Location': '/web/20191009113229/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5310959', 'Set-Cookie': 'JSESSIONID=4594008E80EF728394794EF4302685C3; Path=/; HttpOnly', 'X-Archive-Orig-Vary': 'Accept-Encoding', 'X-Archive-Guessed-Charset': 'UTF-8', 'X-Archive-Orig-Pragma': 'no-cache', 'X-Archive-Orig-Server': 'Apache', 'X-Archive-Orig-Connection': 'close', 'X-Archive-Orig-Content-Type': 'text/html; charset=UTF-8', 'X-Archive-Orig-Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'X-Archive-Orig-Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'X-Archive-Orig-Date': 'Wed,

https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=199709 -- {'Server': 'nginx/1.15.8', 'Date': 'Wed, 09 Oct 2019 11:33:09 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Location': '/web/20191009113306/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=199709', 'Set-Cookie': 'JSESSIONID=4A3378C521C4EA5650CB7868CCD87184; Path=/; HttpOnly', 'X-Archive-Orig-Vary': 'Accept-Encoding', 'X-Archive-Guessed-Charset': 'UTF-8', 'X-Archive-Orig-Pragma': 'no-cache', 'X-Archive-Orig-Server': 'Apache', 'X-Archive-Orig-Connection': 'close', 'X-Archive-Orig-Content-Type': 'text/html; charset=UTF-8', 'X-Archive-Orig-Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'X-Archive-Orig-Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'X-Archive-Orig-Date': 'Wed, 0